<a href="https://colab.research.google.com/github/ParsaRouzrokh/PyTorch_Brain_Tumor_Segmentation_2D/blob/main/Brain_Tumor_Segmentation_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Brain Tumor Segmentation Model Graphical User Interface!**

*Author: Parsa Rouzrokh, MD, MPH*


---




In this notebook, I have created a front-end for one of the brain tumor models that we previously trained. By utilizing Hugging Face's Gradio library, the user interface of our model is now accessible simply by running the following notebook.

You can check out the final version of the interface [here](https://huggingface.co/spaces/Parsa00r/Brain_Seg) without having to run the notebook or install any additional packages!


*   The required packages for running this notebook are available in the requirements.txt file.

*   The information provided by users is being recorded in a publicly accessible notebook called "patients_info.xlsx" which is available on my Google Drive. If you would like access to this file or would like to evaluate the recording process, feel free to make a copy of ["Patients_Info.xlsx"](https://docs.google.com/spreadsheets/d/1uiXs3Ml_gT4d3VxlLiWsOWY6nE6ZLeMF/edit#gid=592561244) into your own Drive and use it *(Ensure about the name of the file "Patients_Info.xlsx")*.



---
Table of Contents:

*   Intro 1: Installing Libraries
*   Intro 2: Importing Necessary Modules
*   Intro 3: Mounting Google Drive
*   Part 1: Model Collection
*   Part 2: Required Files/Folders Collection
*   Part 3: User Interface
    - Part 3.1: Function 1: Img_Hash
    - Part 3.2: Function 2: U_Flex_Predict
    - Part 3.3: Function 3: Submission
    - Part 3.4: Launching Our Demo



# **Intro 1: Installing Libraries**
We start by installing required libraries for this project:


*   gradio (ver 3.45.0)
*   monai
*   gdown

In [1]:
!pip install gradio==3.45.0
!pip install monai
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=4a7950f535ac94f03cf6eaedb8355c4f11db9a21ede97289e2c180fc4fc8a791
  St

# **Intro 2: Importing Necessary Modules**

This notebook is using the mentioned standard/third-party modules:



In [2]:
# Standard library modules
import os
import hashlib

# Third-party library modules
import io
import torch
import gradio as gr
import monai as mn
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import warnings
from PIL import Image
import gdown

warnings.filterwarnings("ignore")

# There is no need for "gpu" in this demo version!
device = torch.device("cpu")

# **Intro 3: Mounting Google Drive**

For accessing to our local database ("Patients_Info"), we should first mount our google drive to our notebook!

Please follow these steps to connect your drive to this notebook:

*   Step 1: Click on "Connect to Google Drive" in the pop-up that appears after running this cell.

*   Step 2: Select the Google account into which you have copied the "Patients_Info.xlsx" file.

*   Step 3: Scroll down in the subsequent window and choose "Allow" to grant access to this notebook!



In [3]:
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# **Part 1: Model Collection**

If you remember, we have trained two models on our MRI 2D slice dataset in the "Brain_Tumor_segmentation" notebook:
*  UFlex: Using flexible UNets from Monai library
*  CustomNet: I wrote it from scratch

For creating our interface, I decide to use our UFlex model and weights. Hence, I saved this model using torch.save to my goole drive and made it public for future use.

So now in this part, let's download this model to our notebook here, and unpack it.

In [4]:
url = 'https://drive.google.com/uc?id=12nY-WdqwHoyUiKVKzpWNjXfuMerTP8Gl'
output = 'UFlex_package.pth'
gdown.download(url, output, quiet=True)
UFlex = torch.load("UFlex_package.pth",map_location=torch.device('cpu'))
model = UFlex['U_Flex_architecture']
weights = UFlex['U_Flex_state_dict']
model.load_state_dict(weights)
model.eval()
model = model.to(device)

# **Part 2: Required Files/Folders Collection**

Before moving to the interface developement, there are two more folders we need to load into this notebook. Same as the model, these two folders are also public and accessible using the links below:
*  [inference_images](https://drive.google.com/file/d/1Cs9ohjiRFa1dYF-8kntn8iatMPmTts9x/view?usp=drive_link)
*  [examples](https://drive.google.com/file/d/1aAhB5IL5AQh-Rv1Yvn_oioYMmnPYjQJ-/view?usp=drive_link)

In [26]:
gdown.download(url= "https://drive.google.com/file/d/1Cs9ohjiRFa1dYF-8kntn8iatMPmTts9x/view?usp=drive_link",
               output = "inference_images.zip", quiet=False, fuzzy=True)
gdown.download(url= "https://drive.google.com/file/d/1aAhB5IL5AQh-Rv1Yvn_oioYMmnPYjQJ-/view?usp=drive_link",
               output = "examples.zip", quiet=False, fuzzy=True)
!unzip -q "inference_images" -d inference_images
!unzip -q "examples" -d examples
os.remove("inference_images.zip")
os.remove("examples.zip")

Downloading...
From: https://drive.google.com/uc?id=1Cs9ohjiRFa1dYF-8kntn8iatMPmTts9x
To: /content/inference_images.zip
100%|██████████| 51.2k/51.2k [00:00<00:00, 57.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aAhB5IL5AQh-Rv1Yvn_oioYMmnPYjQJ-
To: /content/examples.zip
100%|██████████| 151k/151k [00:00<00:00, 19.4MB/s]


# **Part 3: User Interface**

Gradio is a Python library that allows you to create customizable Graphical User Interface (GUI) components for a machine learning model or any Python function.

There are only three required parameters needed to be specified for creating interfaces in gradio:
*   The function to create a GUI for
*   The desired input components
*   The desired output components

So first of all, let's design our graphical demo. Our demo should include these parts:



1.   Title and description of our model
2.   Getting patient's information
3.   Getting patient's mri slice
4.   Submission of the info and showing the tumor-identified version to user

Below, you can see the defined functions before starting the demo cell. But maybe it's better to design your demo first and then, define your functions.

For this demo, we define three functions:

*   img_hash: A function to convert images to they hash forms for more convenient recording.
*   U_Flex_predict: An inference function of our model
*   submission: A function responsible for recording all the given info, and returning the output of our model to user.

Now, everything is set! The next three sections (Part 3.1, Part 3.2, and Part 3.3), are all about the functions. So feel free to navigate to Part (3.4) for designing part before checking out the functions.

# Part 3.1: Function 1: Img_Hash

Since we want to record the input and output slices, along with the patients' information in an excel file, I decide to convert these arrays to their hash forms.


In [6]:
def Img_Hash (array):
  return hashlib.sha1(array).hexdigest()

# Part 3.2: Function 2: U_Flex_Predict

We have the best version of our model right now. So it's ready for the inference phase. In this function, we perform some pre/post processings according to the needs of our model, and our demo.   


In [7]:
def U_Flex_Predict(input_image):
  input_image = input_image.convert("L")
  image_array =  np.array(input_image)
  inference_transforms = mn.transforms.Compose([
    mn.transforms.EnsureChannelFirst(channel_dim="no_channel"),
    mn.transforms.Resize(spatial_size= image_array.shape),
    mn.transforms.ScaleIntensity(minv=0, maxv=1),
    mn.transforms.ToTensor()
  ])
  image = inference_transforms(image_array)
  with torch.no_grad():
    image = image.unsqueeze(0).to(device)
    output = model(image)
  infer_img = image.to(device)
  infer_out = output.to(device)
  plt.imshow(infer_img[0][0].detach().cpu(), cmap='gray')
  plt.imshow(infer_out.squeeze(0).argmax(dim=0).detach().cpu(),interpolation='nearest', alpha=0.5)
  plt.axis('off')
  plot_bytes = io.BytesIO()
  plt.savefig(plot_bytes, format='jpg')
  plot_bytes.seek(0)
  result_image = Image.open(plot_bytes)
  result_hash = Img_Hash(np.array(result_image))
  input_hash = Img_Hash(np.array(input_image))
  result_dict = {"result_image": result_image, "result_hash": result_hash, "input_hash": input_hash}
  return result_dict

# Part 3.3: Function 3: Submission
In this part, we provide a function which is called after the user clicks on the "submit" button. It includes various parts listed below:
*   Inputs sanity checks
*   Showing the tumor-identified version of input MRI slice
*   Recording the patients info, along with input and output hashed images in our local database (Patients_Info.xlsx).


In [8]:
def Submission(Age,Drug,PMH,Symptoms,Family,Other_Info, Drug_Yes,Family_Yes,Other_Yes,PMH_others, input_image):
  necessary_fields = [Age, Drug, PMH, Symptoms, Family, Other_Info]
  filled_fields = sum(bool(field) for field in necessary_fields)

  # Sanity check part 1:

  if Drug == "Yes" and not Drug_Yes:
    raise gr.Error("Please enter the name of your medications before submitting!")
  elif Drug == "No" and Drug_Yes:
    raise gr.Error("Please leave the 'Medications or Treatments' field blank if you have no history of taking medications!")
  if "Others" in PMH and not PMH_others:
    raise gr.Error("Please enter the name of your physical conditions which are not listed above before submitting!")
  if "Others" not in PMH and PMH_others:
    raise gr.Error("You have to check 'Others' if you want to add extra conditions")
  elif "Nothing" in PMH and PMH_others:
    raise gr.Error("Please leave the 'Other Conditions' field blank if you have no other conditions!")
  elif "Nothing" in PMH and len(PMH) > 1:
    raise gr.Error("You can not select 'Nothing' along with other options. Please try again.")
  if Family == "Yes" and not Family_Yes:
    raise gr.Error("Please enter your familiar history before submitting!")
  elif Family == "No" and Family_Yes:
    raise gr.Error("Please leave the 'Familial History' field blank if you have no familial history!")
  if "No" in Symptoms and len(Symptoms) > 1:
    raise gr.Error("You can not select No along with other symptoms!")
  if Other_Info == "Yes" and not Other_Yes:
    raise gr.Error("Please enter any other relevant factors or medical information that\
    you believe may be important for us before submitting!")
    error +=1
  elif Other_Info == "No" and Other_Yes:
    raise gr.Error("Please leave the 'Other info' field blank if you have no other info!")

  if not input_image:
    raise gr.Error("Please upload your brain MRI slice before submission!")

  # sanity check part 2:

  sample_1_array = np.array(Image.open("examples/sample_image1.jpg").convert("L"))
  sample_2_array = np.array(Image.open("examples/sample_image2.jpg").convert("L"))
  sample_3_array = np.array(Image.open("examples/sample_image3.jpg").convert("L"))
  sample_1_mask_array = np.array(Image.open("examples/sample_image1_mask.jpg").convert("L"))
  sample_2_mask_array = np.array(Image.open("examples/sample_image2_mask.jpg").convert("L"))
  sample_3_mask_array = np.array(Image.open("examples/sample_image3_mask.jpg").convert("L"))

  if (Img_Hash(sample_1_array) == Img_Hash(np.array(input_image.convert("L"))))\
   or (Img_Hash(sample_2_array) == Img_Hash(np.array(input_image.convert("L"))))\
   or (Img_Hash(sample_3_array) == Img_Hash(np.array(input_image.convert("L"))))\
   or (Img_Hash(sample_1_mask_array) == Img_Hash(np.array(input_image.convert("L"))))\
   or (Img_Hash(sample_2_mask_array) == Img_Hash(np.array(input_image.convert("L"))))\
   or (Img_Hash(sample_3_mask_array) == Img_Hash(np.array(input_image.convert("L")))):
       raise gr.Error("You can not submit the examples as your MRI slice! Please \
       clear this image and choose your own!")

  # Inferring the input image and recording the info to our local database!

  results = U_Flex_Predict(input_image)
  if filled_fields == len(necessary_fields) and input_image:
      records = [Age,Drug,Drug_Yes,PMH,PMH_others,Symptoms,Family,Family_Yes,\
                 Other_Info,Other_Yes,results['input_hash'],results['result_hash']]
      !cp '/content/drive/MyDrive/Patients_Info.xlsx' Patients_Info.xlsx
      df = pd.read_excel('Patients_Info.xlsx')
      df = df.append(pd.Series(records, index=df.columns), ignore_index=True)
      df.to_excel('Patients_Info.xlsx', index=False)
      !cp 'Patients_Info.xlsx' '/content/drive/MyDrive/Patients_Info.xlsx'
      !rm 'Patients_Info.xlsx'

      gr.Info("Thank you for your submission! Your information has been received.\
      If you want to do the entire test for another person, click the 'Clear' button and repeat the process.")

      return(results['result_image'])

  else:
      raise gr.Error(f" Please fill all the fields before submission")

# **Part 3.4: Launching Our Demo**

In this part, we initiate our demo and launch it using our functions defined above. Our demo includes:
1.   Title and description of our model
2.   Creating different elements for making selections, using gardio components
3.   Showing some examples and the outputs
4.   Creating an image component that can be used to upload images (as an input) or display images (as an output)
5.   Creating a submission button and link it with the function defined above

After running the following cell, the GUI we have just designed will be ready. You can access it through the provided link below.
Please note that the displayed inputs and outputs in the GUI are examples and cannot be used as actual input. However, the "inference_images" folder you downloaded earlier contains real 2D FLAIR images that you can use to evaluate the model/demo.
Also, you can checkout your local database in your drive and evaluate the recordings!



In [27]:
with gr.Blocks() as demo:

  # 1. Title and description of our model:

  gr.Markdown("<center><b><font size='10'>Welcome to the Brain Tumor Segmentation Lab!</font></b></center>")
  gr.Markdown("<center><b><font size ='5'>In this page, you can upload your 2D \
  Brain FLAIR MRI slice, and detect the tumor location in a blink of an eye!</font></b></center>")
  gr.Markdown("<center><font size ='3'>Please follow the instructions in order \
  and provide feedback by contacting me at <a href='mailto:parsa.rouzrokh97@gmail.com'>\
  parsa.rouzrokh97@gmail.com</a> </font></center>")
  gr.Markdown("<font size ='2'><b>Step1:</b></font> We need some information \
  about your condition before uploading your MRI slice.")
  gr.Markdown("* Rest assured that all your information is securely stored and \
  treated with the utmost confidentiality to ensure your privacy.")
  gr.Markdown("* Questions marked with an asterisk (*) are necessary to answer before submission.")

  # 2. Creating different elements for making selections, using gardio components

  with gr.Row():
      with gr.Column():
        Age = gr.Radio(
        ["under 20", "20-30", "30-40","40-50","50-60","More than 60"],
        label="*1.Please tell us your age range:")
        Drug = gr.Radio(
          ["Yes","No"],
          label="*2.Are you currently taking any medications or undergoing any \
          other medical treatments?",
          info ="If you click 'Yes', please tell us the medications or treatments\
          you have in a text box below!"
      )
        Drug_Yes = gr.Textbox(placeholder="Please fill this if you check 'Yes' above!",
                                label="Medications or Treatments:")
        PMH = gr.CheckboxGroup(
        ["Nothing","Hypertension","Diabetes","Seizure","Others"],
        label="*3.What are your known medical conditions or history of neurological disorders?",
        info = "If you click 'others', please tell us the conditions you have in a text box below!")
        PMH_others = gr.Textbox(placeholder="Please fill this if you check 'others' above!",
                                label="Other Conditions:")
      with gr.Column():
        Symptoms = gr.CheckboxGroup(
          ['No','persistent headaches','seizures','changes in vision'],
          label="*4.Have you experienced any recent symptoms related to the brain,\
          such as persistent headaches, seizures, or changes in vision?")
        Family = gr.Radio(["Yes","No"],
        label="*5.Is there a family history of brain\
         tumors or other neurological conditions?",
         info = "If you click 'Yes', please tell us the history in a text box below!"
         )
        Family_Yes = gr.Textbox(placeholder="Please fill this if you check 'Yes' above!",
                                label="Family History:")
        Other_Info = gr.Radio(["Yes","No"],
        label="*6.Are there any other relevant factors or medical information that \
        you believe may be important for us to know about?",
        info = "If you click 'Yes', please tell us the details in a text box below!"
         )
        Other_Yes = gr.Textbox(placeholder="Please fill this if you check 'Yes' above!",
                                label="Other Info")

  gr.Markdown("<font size ='2'><b>Step2:</b></font> Now it's time to upload the MRI slice!")
  gr.Markdown("* Upload your picture by simply clicking on the left box and selecting\
  the MRI image from your local system.")
  gr.Markdown("* If you fill all of the blanks above, You will be able to do the submission\
  and get your tumor identified version in the right box!")
  gr.Markdown("* We recommend that you crop the identification part of your image\
  by selecting the 'brain volume' using the 'edit' button located at the top right of the uploaded picture.")
  gr.Markdown("* For better accuracy, please ensure that you upload your image in\
  the same orientation as the examples we have provided for you below.")
  gr.Markdown("<center><font size ='3'> Here are three example MRI 2D slices along\
  with the tumor location identified versions next to them!</font></center>")

  # 3. Showing some examples and the outputs

  gr.Gallery(["examples/sample_image1.jpg",
              "examples/sample_image1_mask.jpg",
                "examples/sample_image2.jpg",
                "examples/sample_image2_mask.jpg",
                "examples/sample_image3.jpg",
              "examples/sample_image3_mask.jpg"],
             label = "Examples:", object_fit="contain", height="auto",columns=[6], rows=[1])

  # 4. Creating an image component that can be used to upload images (as an input) or display images (as an output)

  with gr.Row():
    with gr.Column():
      gr.Markdown("Upload your image here!")
      input_image = gr.Image(type='pil')

    with gr.Column():
      gr.Markdown("This is where the output image appears!")
      output_image = gr.Image(type='pil',container=False)

  # 5. Creating a submission button and link it with the function defined above

  Submit = gr.Button(value="Submit!",interactive=True)
  Submit.click(Submission,
               inputs=[Age,Drug,PMH,Symptoms,Family,Other_Info,Drug_Yes,Family_Yes,Other_Yes,PMH_others,input_image],
               outputs= output_image)
  clear = gr.ClearButton([Age,Drug,PMH,Symptoms,Family,Other_Info,Drug_Yes,Family_Yes,Other_Yes,PMH_others,input_image,output_image])
  if __name__ == "__main__":
      demo.queue()
      demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ce4872214a7bb905cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Thank you for accompanying me on this journey! I would appreciate it if you could share your feedback with me via email at parsa.rouzrokh97@gmail.com.

See you soon!